In [1]:
import random
from datetime import date, timedelta
from datetime import datetime as dt
import pandas as pd
import itertools

In [2]:
def getRandomTime(min_x, max_x, start_time):
    start_time = dt.strptime(start_time, "%Y-%m-%dT%H:%M:%S:%f")
    # min and max duration of the activity %H:%M:%S
    min_duration = dt.combine(date.min, dt.strptime(min_x, "%H:%M:%S:%f").time())
    max_duration = dt.combine(date.min, dt.strptime(max_x, "%H:%M:%S:%f").time()) 
    # get a ramdom millisec value between min and max duration
    millisec = random.randint(0, int(((max_duration - min_duration).total_seconds())*1000))
    # sum the start date and the calculated random value
    min_x = dt.strptime(min_x, '%H:%M:%S:%f')
    b = timedelta(hours= min_x.hour, minutes = min_x.minute, seconds = min_x.second, milliseconds = min_x.microsecond)
    random_duration = start_time + timedelta(milliseconds = millisec) + b
    return (random_duration.strftime('%Y-%m-%dT%H:%M:%S:%f'), millisec+int(b.total_seconds()*1000))    

In [3]:
def addDelay(time):
    #delay tra le attivazioni, tra i 200 e i 400 secondi
    delay = random.randint(200, 400)
    return (dt.strptime(time, '%Y-%m-%dT%H:%M:%S:%f') + timedelta(seconds=delay)).strftime('%Y-%m-%dT%H:%M:%S:%f')

In [4]:
def createEventLog(rep, shuffle):
    start_time = dt(2022, 1, 1, 7, 0, 0, 0).strftime('%Y-%m-%dT%H:%M:%S:%f')
    day = []

    for j in range(rep):
        # get a random time to wake up, between 07:00 and 08:00
        # start time is 07:00, so the function takes a range of 1 h
        wake_up = getRandomTime("00:00:00:00", "01:00:00:00", start_time)
        wake_up_complete = addDelay(wake_up[0])
        day.append([j, "Wake_up", wake_up[0], wake_up_complete])

        if shuffle == True:
            # we can execute or not execute the get_dressed
            gt = random.choice([True, False])
            # gt = True
            if gt == True:
                # get dressed has a duration between 15 and 30 minutes
                get_dressed = addDelay(wake_up_complete)
                get_dressed_complete = getRandomTime("00:15:00:00", "00:30:00:00", get_dressed)
                day.append([j, "Get_dressed", get_dressed, get_dressed_complete[0]])     
            
            temp = []
            temp_breakfast = []
            temp_bed = []
            temp_breakfast.append(["Have_breakfast", "00:20:00:00", "00:45:00:00"])
            temp_bed.append(["Make_bed", "00:05:00:00", "00:10:00:00"])
            temp.append(temp_breakfast)
            temp.append(temp_bed)
            random.shuffle(temp)
            row = list(itertools.chain.from_iterable(temp))
            l = len(day)
            start_x = addDelay(day[l-1][3])
            # row[0]
            a = getRandomTime(row[0][1],row[0][2], start_x)
            day.append([j, row[0][0], start_x, a[0]])

            row1_delay = addDelay(a[0])
            # row[1]
            b = getRandomTime(row[1][1],row[1][2], row1_delay)
            day.append([j, row[1][0], row1_delay, b[0]])

            # go bathroom has a duration between 15 and 25 minutes
            go_bathroom = addDelay(b[0])
            go_bathroom_complete = getRandomTime("00:15:00:00", "00:25:00:00", go_bathroom)
            day.append([j, "Go_bathroom", go_bathroom, go_bathroom_complete[0]])
        
            if gt == False:
                # get dressed has a duration between 15 and 30 minutes
                get_dressed = addDelay(go_bathroom_complete[0])
                get_dressed_complete = getRandomTime("00:15:00:00", "00:30:00:00", get_dressed)
                day.append([j, "Get_dressed", get_dressed, get_dressed_complete[0]]) 
        else:   
            # have breakfast has a duration between 20 to 45 minutes
            have_breakfast = addDelay(wake_up_complete)
            have_breakfast_complete = getRandomTime("00:20:00:00", "00:45:00:00", have_breakfast)
            day.append([j, "Have_breakfast", have_breakfast, have_breakfast_complete[0]])

            # go bathroom has a duration between 15 and 25 minutes
            go_bathroom = addDelay(have_breakfast_complete[0])
            go_bathroom_complete = getRandomTime("00:15:00:00", "00:25:00:00", go_bathroom)
            day.append([j, "Go_bathroom", go_bathroom, go_bathroom_complete[0]])

            # make bed has a duration between 5 and 10 minutes
            make_bed = addDelay(go_bathroom_complete[0])
            make_bed_complete = getRandomTime("00:05:00:00", "00:10:00:00", make_bed)
            day.append([j, "Make_bed", make_bed, make_bed_complete[0]])

            # get dressed has a duration between 15 and 30 minutes
            get_dressed = addDelay(make_bed_complete[0])
            get_dressed_complete = getRandomTime("00:15:00:00", "00:30:00:00", get_dressed)
            day.append([j, "Get_dressed", get_dressed, get_dressed_complete[0]])

        # go out is just a few minutes activity, that is the delay, between 200 and 400 seconds
        l = len(day)
        go_out = addDelay(day[l-1][3])
        go_out_complete = addDelay(go_out)
        day.append([j, "Go_out", go_out, go_out_complete])

        start_time_temp = dt.strptime(start_time, "%Y-%m-%dT%H:%M:%S:%f")
        start_time = (start_time_temp + timedelta(days=1)).strftime('%Y-%m-%dT%H:%M:%S:%f')
    return day

In [5]:
from pm4py.objects.log.exporter.xes import exporter as xes_exporter
from pm4py.objects.conversion.log import converter as log_converter
import pm4py
import os

PATH = '../Scenario3' 
if not os.path.exists(PATH):
    os.makedirs(PATH)

cols = ['case_id','activity', 'start_timestamp', 'complete_time']

#createEventLog(n of traces, shuffle)
log = createEventLog(1000, False)
df = pd.DataFrame(log, columns = cols)
event_log = pm4py.format_dataframe(df, case_id='case_id', activity_key='activity', timestamp_key='complete_time', start_timestamp_key='start_timestamp', timest_format="%Y-%m-%dT%H:%M:%S:%f")
pm4py.write_xes(event_log, '../Scenario3/logNormal.xes')

log2 = createEventLog(1000, True)
df2 = pd.DataFrame(log2, columns = cols)
event_log2 = pm4py.format_dataframe(df2, case_id='case_id', activity_key='activity',  timestamp_key='complete_time', start_timestamp_key='start_timestamp', timest_format="%Y-%m-%dT%H:%M:%S:%f")
pm4py.write_xes(event_log2, '../Scenario3/logShuffle.xes')


yes


c:\Users\gdfe\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
exporting log, completed traces :: 100%|██████████| 1000/1000 [00:00<00:00, 1108.32it/s]
